In [1]:
#Import Module
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/0gE1d4-MDxpCRWZecTGAP8OVhnBSStlDF6id8F_mVN47jt3EM8HXzdw

Successfully saved authorization token.


In [3]:
#Input Variabels
start = input("Tanggal Awal:")
finish = input("Tanggal Akhir:")
bounds = geemap.shp_to_ee(input("Lokasi shp Area:"))


#Image Collection
#Image Colection1
collections = input("Input Images Collection Link:")
imagescol = ee.ImageCollection(collections).filterDate(start, finish).filterBounds(bounds).filter('CLOUDY_PIXEL_PERCENTAGE < 5').median()

print('images collection telah dibuat:', 'imagescol')

Tanggal Awal:2019-03-01
Tanggal Akhir:2019-08-30
Lokasi shp Area:D:\Cloud Computing\GEE\SHP\TN\TNBTS (dari Kang Hardjono)\ADM_TNBTS.shp
Input Images Collection Link:COPERNICUS/S2
images collection telah dibuat: imagescol


In [4]:
#input Data Vector
Vector = geemap.shp_to_ee(input("Masukkan Lokaasi Vector:")) #Menambahkan Data Vector
print("Vector telah dibuat")

Masukkan Lokaasi Vector:D:\Cloud Computing\GEE\SHP\TN\TNBTS (dari Kang Hardjono)\RESORT_TNBTS.shp
Vector telah dibuat


In [5]:
#Indeks Vegetasi Function
def indeksVegetasi(img):
    rumus = input('Masukkan Formula:')
    indeks = img.expression(rumus,
                           {'B1': img.select('B1'),
                            'B2': img.select('B2'),
                            'B3': img.select('B3'), 'B4': img.select('B4'), 'B5': img.select('B5'), 'B6': img.select('B6'),
                            'B7': img.select('B7'), 'B8': img.select('B8'), 'B9': img.select('B9'), 'B10': img.select('B10'),
                            'B11': img.select('B11'), 'B12': img.select('B12'), 'B8A': img.select('B8A'), 
                            'QA10': img.select('QA10'), 'QA20': img.select('QA20'), 'QA60': img.select('QA60')})
    return indeks


In [6]:
NDVI = indeksVegetasi(imagescol)

Masukkan Formula:(B8-B4)/(B8+B4)


In [7]:
NBR = indeksVegetasi(imagescol)

Masukkan Formula:(B8-B12)/(B8+B12)


In [16]:
#Visualisasi Overlay Hasil
Map = geemap.Map()
Map.centerObject(bounds, zoom=12)

Map.addLayer(imagescol.clip(bounds),
            {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500}, #Menampilkan Layer Images Collection
            name='ImagesCllection')
Map.addLayer(NDVI.clip(bounds), {'palette': ['red', 'yellow', 'green']}, #Menambahkan Layer Indeks NDVI
                                name='NDVI')
Map.addLayer(NBR.clip(bounds), {'palette': ['red', 'yellow', 'green']}, #Menambahkan Layer Indeks NBR
                                name='NBR')
Map.addLayer(Vector, {}, name='Resort')
Map

Map(center=[-8.033989020419346, 112.93390157219612], controls=(WidgetControl(options=['position'], widget=HBox…

In [17]:
def makeSample(id):
    variable = Map.draw_features
    samplesfeature = ee.FeatureCollection(variable)
    poly = ee.Geometry.MultiPolygon(samplesfeature.geometry().coordinates())
    feature = ee.Feature(poly, {'class':id})
    
    return feature

In [18]:
Pasir = makeSample(3)

In [20]:
trainingsamples = ee.FeatureCollection([Semak, Hutan, Pasir])

In [65]:
bands = ['B8', 'B4', 'B3', 'B2']

In [75]:
#Extract Pixel Value by Features for Training
training = imagescol.select(bands).sampleRegions(**{'collection': trainingsamples,
                                   'properties': ['class'],
                                   'scale':10})

In [76]:
#Define Classifier and Train
DTC = ee.Classifier.randomForest(**{'bagFraction':0.2, 
                                   'numberOfTrees':200}).train(training, 'class', bands)

In [77]:
#Classsify
classified = imagescol.select(bands).clip(bounds).classify(DTC)

In [88]:
Map = geemap.Map()

Map.centerObject(bounds, zoom=12)
Map.addLayer(imagescol.clip(bounds),
            {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500}, #Menampilkan Layer Images Collection
            name='ImagesCllection')

Map.addLayer(classified.clip(bounds), {'min':1,
                         'max':3,
                         'palette': ['yellow', 'green', 'red']}, "Hasil Klasifikasi")

Legend_dict = {'Semak': '#f2f414','Hutan': '#2b814b', 'Pasir':'#f45823'}
Map.add_legend('Legend PL', Legend_dict)



Map

Map(center=[-8.033989020419346, 112.93390157219612], controls=(WidgetControl(options=['position'], widget=HBox…